<br>Conteúdo:
* [Analisando os Dados](#1)
* [Analisando os dados com base na linha do tempo](#2)
* [Análise nível Loja](#3)
* [Analise nível Departamento](#4)
* [Análise de Loja por tipo e por Ano/Mês](#5) 
* [Análise MarkDown](#6)
* [Previsão por Data e Loja](#7)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import requests
import csv
warnings.filterwarnings('ignore')

In [3]:
# Arquivo com informações de loja
df_store = pd.read_csv('https://raw.githubusercontent.com/rbessa79/tcc_pucminas/main/data/stores-data-set.csv')

# Arquivo com informações complementares
df_feature = pd.read_csv("https://raw.githubusercontent.com/rbessa79/tcc_pucminas/main/data/Features-data-set.csv", parse_dates=['Date'])

# Arquivo com informações de vendas
df_sales = pd.read_csv("https://raw.githubusercontent.com/rbessa79/tcc_pucminas/main/data/sales-data-set.csv", parse_dates=['Date'])

ConnectionResetError: [WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto

In [ ]:
# df_store = pd.read_csv("C:\\Users\\rbessa\\Downloads\\archive\\stores data-set.csv")
# df_feature = pd.read_csv("C:\\Users\\rbessa\\Downloads\\archive\\Features data set.csv",parse_dates=["Date"])
# df_sales = pd.read_csv("C:\\Users\\rbessa\\Downloads\\archive\\sales data-set.csv",parse_dates=["Date"])


<a id="1"></a> <br>
# Analisando os dados

verificando as tabelas e informações básicas
* Tipo dos dados
* Número de dados (linhas e colunas)
* Dados nulos
* Algumas linhas para entender os dados

In [ ]:
print(df_store.info())
print(df_store.head())

In [ ]:
print(df_feature.info())
print(df_feature.head())

In [ ]:
print(df_sales.info())
print(df_sales.head())

Analise prelimiar dos dados, chego à conclusão de que podemos agrupar dados com base no tempo, loja, departamento.
Também podemos verificar como feriados, temperatura, preço do combustível, CPI, desemprego e tipo de loja afetam as vendas.

<a id="2"></a> <br>
# **Analisando os dados com base na linha do tempo**

* Jutando as tabelas Features e Sales baseados na data 
* Converter vendas semanais em milhões
* Modificando a coluna conforme o dia é feriado ou não

In [ ]:
data_date = df_feature.groupby("Date").agg({"Temperature":"mean"
                                            ,"Fuel_Price":"mean"
                                            ,"IsHoliday":"sum"
                                            ,"CPI":"mean"
                                           ,"Unemployment":"mean"})
data_date = data_date.sort_index()
temp_date_data = data_date[:'2012-12-10']

data_sales_date = df_sales.groupby("Date").agg({"Weekly_Sales":"sum"})
data_sales_date.sort_index(inplace=True)
data_sales_date.Weekly_Sales = data_sales_date.Weekly_Sales/1000000
data_sales_date.Weekly_Sales = data_sales_date.Weekly_Sales.apply(int)
data = pd.merge(data_sales_date, temp_date_data, left_index=True,right_index=True, how='left')
data["IsHoliday"] = data["IsHoliday"].apply(lambda x: True if x == 45.0 else False )


In [ ]:
print(data.describe())

In [ ]:
plt.style.use('fivethirtyeight')
fig, ax = plt.subplots(5,1,figsize=(15,10),sharex=True) 
data["Weekly_Sales"].plot(ax=ax[0],title="Weekly Sales/sales on Holiday")
data[data.IsHoliday==True]["Weekly_Sales"].plot(marker="D",ax=ax[0],legend="Holiday Week sale")
data["Temperature"].plot(ax=ax[1], title="Temperature")
data["Fuel_Price"].plot(ax=ax[2],title="Fuel_Price")
data["CPI"].plot(ax=ax[3],title="CPI")
data["Unemployment"].plot(ax=ax[4],title="Unemployment")


* Nov- Dec mostram aumento nas vendas semanais, mas ao longo do ano não há crescimento. 
* Semanas que antecedem feriados mostram picos de vendas
* O Preço dos Combustíveis e o Índice de Preços ao Consumidor apresentaram crescimento ao longo do ano.
* O desemprego diminuiu ano após ano.
* A temperatura está mostrando uma caminhada aleatória -- VERIFICAR


In [ ]:
sns.heatmap(data.corr(),annot=True)

* As vendas semanais não mostram nenhuma correlação alta com nenhum outro parâmetro.
* CPI e Desemprego mostram correlação negativa, por outro lado, o preço do combustível mostra correlação positiva.
* O desemprego e o preço do combustível também estão negativamente correlacionados.

In [ ]:
data_sales_month = data.groupby(data.index.month).agg({"Weekly_Sales":"sum"})
plt.figure(figsize=(10, 5))
sns.barplot(x=data_sales_month.index,y=data_sales_month.Weekly_Sales)
plt.title("Month wise Sales")
plt.xlabel("Month")
plt.ylabel("Sales")

* Pode ser o suficiente para comprar durante as ofertas e feriados (vendas na Black Friday, Natal, Ano Novo) e as vendas posteriores caem e, em seguida, ganham impulso novamente em alguns meses.
* Em novembro, também as pessoas mostram que menos compras podem ser um período de espera para as próximas vendas. --VERIFICAR
* Os dados completos para dezembro de 2013, acho que não estão disponíveis, portanto, mesmo após o pico em dezembro, as vendas não subiram muito.

In [ ]:
data_sales_year = data.groupby(data.index.year).agg({"Weekly_Sales":"sum"})

sns.barplot(x=data_sales_year.index,y=data_sales_year.Weekly_Sales)
plt.title("Year wise Sales")
plt.xlabel("Year")
plt.ylabel("Sales")


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(data["Weekly_Sales"], period=45)

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(decomposition.trend)
plt.plot(decomposition.seasonal)
plt.plot(decomposition.resid)
plt.legend(["Trend", "Seasonal","Resid"], loc ="upper right")

* Aqui mostra uma tendência lateral

<a id="3"></a> <br>
# **Análise nível Loja**

vamos explorar o padrão armazenado para dados de vendas semanais

In [ ]:
data_Store = df_feature.groupby("Store").agg({"Temperature":"mean","Fuel_Price":"mean","IsHoliday":"sum"})

temp_store = df_sales.groupby("Store").agg({"Weekly_Sales":"sum"})
temp_store.Weekly_Sales = temp_store.Weekly_Sales/1000000
temp_store.Weekly_Sales = temp_store.Weekly_Sales.apply(int)
data_Store.set_index(np.arange(0,45),inplace=True)
df_store["temp"] = data_Store.Temperature
df_store["Fuel_Price"] = data_Store.Fuel_Price
df_store["holiday"] = data_Store.IsHoliday
df_store["Weekly_Sales"] = temp_store.Weekly_Sales

In [ ]:
df_store.describe()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15, 4))
sns.countplot(df_store.Type,ax=ax[0])
sns.swarmplot(data = df_store,y="Size",x="Type",ax=ax[1])

sns.boxplot(data = df_store,y="Weekly_Sales",x="Type",ax=ax[2])



* Contagem de lojas está na ordem de A, B e menos é C
* O tamanho de Lojas tipo A armazena a maioria dos itens seguidos por B e depois por C --VERIFICAR
* Mesmo depois de menos contagem de lojas e menor tamanho B, tem mais vendas semanais do que A e C, pelo menos.

<a id="4"></a> <br>
# **Analise nível Departamento**

Uma pequena investigação por departamento

In [ ]:
len(df_sales["Dept"].unique())

In [ ]:
data_Dept = df_sales.groupby("Dept").agg({"Weekly_Sales":"sum"})
data_Dept.Weekly_Sales = data_Dept.Weekly_Sales/10000
data_Dept.Weekly_Sales = data_Dept.Weekly_Sales.apply(int)
data_Dept.sort_values(by="Weekly_Sales")

In [ ]:
fig1, ax1 = plt.subplots(figsize=(15, 4))
plt.vlines(x=data_Dept.index, ymin=0, ymax=data_Dept['Weekly_Sales'], color='skyblue')
plt.plot(data_Dept.index,data_Dept['Weekly_Sales'], "o")
plt.title("Departmentwise Sales")
plt.ylabel("Sales")
plt.xlabel("Department")

* Aqui podemos ver que os departamentos 1-15 e 90-95 mostraram vendas semanais mais altas.
* Alguns departamentos como 38,40,72 mostraram vendas semanais mais altas.

<a id="5"></a> <br>
# Análise de Loja por tipo e por Ano/Mês 

In [ ]:
sales_date_store = df_sales.groupby(["Date","Store"]).agg({"Weekly_Sales":"sum"})
sales_date_store.sort_index(inplace=True)
sales_date_store.Weekly_Sales = sales_date_store.Weekly_Sales/10000
sales_date_store.Weekly_Sales = sales_date_store.Weekly_Sales.apply(int)
data_table = pd.merge(df_feature,sales_date_store ,  how='left', on=["Date","Store"])
data_table = pd.merge(data_table,df_store[["Store","Type"]] ,  how='left', on=["Store"])
data_table.head(20)
data_train = data_table[data_table.Weekly_Sales.notnull()]
data_test = data_table[data_table.Weekly_Sales.isnull()]

In [ ]:
plt.figure(figsize=(15, 5))
sns.barplot(x=data_train.Date.dt.year, y=data_train.Weekly_Sales,hue=data_train.Type)

* As lojas do tipo A & B apresentaram uma ligeira queda ano a ano na base de vendas.
* A loja C apresentou um leve aumento nas vendas.

In [ ]:
plt.figure(figsize=(15, 7))
sns.barplot(x=data_train.Date.dt.month, y=data_train.Weekly_Sales,hue=data_train.Type)

* As lojas do tipo A e B mostraram pico em novembro e dezembro, principalmente devido ao feriado, enquanto janeiro mostrou queda nas vendas.

* As loja do tipo C são consistentes ao longo de cada mês e não apresentaram muitos desvios.
   

<a id="6"></a> <br>
# Análise MarkDown

In [ ]:
plt.figure(figsize=(15,4))
train_markdown = data_table[data_table.MarkDown2.notnull()]
train_markdown = train_markdown.groupby("Date").agg({"MarkDown1":"mean","MarkDown2":"mean","MarkDown3":"mean","MarkDown4":"mean","MarkDown5":"mean"})


plt.plot(train_markdown.index,train_markdown.MarkDown1)
plt.plot(train_markdown.index,train_markdown.MarkDown2)
plt.plot(train_markdown.index,train_markdown.MarkDown3)
plt.plot(train_markdown.index,train_markdown.MarkDown4)
plt.plot(train_markdown.index,train_markdown.MarkDown5)
plt.title("Timeline Markdown")
plt.ylabel("Markdown")
plt.xlabel("Date")

In [ ]:
train_markdown.hist(figsize=(10,8),bins=6)
plt.tight_layout()
plt.show()

In [ ]:
train_markdown_month = train_markdown.groupby(train_markdown.index.month).agg({"MarkDown1":"mean","MarkDown2":"mean","MarkDown3":"mean","MarkDown4":"mean","MarkDown5":"mean"})

train_markdown_month.plot(kind='bar', stacked=True,figsize=(15,6))
plt.title("Stacked Monthwise Morkdown")
plt.ylabel("Markdown")

In [ ]:
train_markdown_1 = data_table[data_table.MarkDown2.notnull()]
train_markdown_type = train_markdown_1.groupby("Type").agg({"MarkDown1":"mean","MarkDown2":"mean","MarkDown3":"mean","MarkDown4":"mean","MarkDown5":"mean"})

train_markdown_type.plot(kind='bar', stacked=True,figsize=(10,4))
plt.title("Stacked StoreType Wise")
plt.ylabel("Markdown")

<a id="7"></a> <br>
# **Previsão por Data e Loja**

In [ ]:

from fancyimpute import IterativeImputer
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR, LinearSVR, NuSVR
from sklearn.linear_model import ElasticNet, Lasso, RidgeCV,LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb

**Limpeza e pré-processamento**

In [ ]:
def createmodel(data,cols):
    for col in cols:
        one_hot = pd.get_dummies(data[col],prefix=col)
        data = data.join(one_hot)
        data.drop(col,axis = 1,inplace=True)
    
    return data
        

In [ ]:

# Imputando valores faltantes
itt = IterativeImputer()
df = itt.fit_transform(data_table[["MarkDown1","MarkDown2","MarkDown3","MarkDown4","MarkDown5"]]) 
data_table.MarkDown1 = df[:,0]
data_table.MarkDown2 = df[:,1]
data_table.MarkDown3 = df[:,2]
data_table.MarkDown4 = df[:,3]
data_table.MarkDown5 = df[:,4]

data_table['CPI'].fillna((data_table['CPI'].mean()), inplace=True)
data_table['Unemployment'].fillna((data_table['Unemployment'].mean()), inplace=True)
data_table['IsHoliday'] = data_table['IsHoliday'].map({True:0,False:1})

# criando nova coluna
data_table["Month"] = data_table.Date.dt.month
data_table["Year"] = data_table.Date.dt.year
data_table["WeekofYear"] = data_table.Date.dt.weekofyear
data_table.drop(['Date'],axis=1,inplace=True)

# Criando modelos fora da coluna categoria
data_table = createmodel(data_table,["Type","Month","Year","WeekofYear"])



In [ ]:
data_table.columns

**Divisão de Dados**

In [ ]:
data_train = data_table[data_table.Weekly_Sales.notnull()]
data_test = data_table[data_table.Weekly_Sales.isnull()]
X = data_train.drop('Weekly_Sales', axis=1)
y = data_train['Weekly_Sales']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

**Criação de modelo básico**

In [ ]:



classifiers = [
    LinearRegression(),
    ElasticNet(),
    RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]),
    KernelRidge(alpha=0.6, kernel='polynomial', degree=3, coef0=2.5),
    Lasso(alpha =16, random_state=100),
    ElasticNet(alpha=0.8),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor(),
    AdaBoostRegressor(),
    SVR(), 
    LinearSVR(), 
    NuSVR(),
    xgb.XGBRegressor(),
    lgb.LGBMRegressor()
    ]

name = []
score = []
models = []
rmse = []
i = 0
for classifier in classifiers:
    classifier.fit(X_train, y_train)   
    name.append(type(classifier).__name__)
    score.append(classifier.score(X_test, y_test))
    models.append(classifier)
    rmse.append(np.sqrt(mean_squared_error(classifier.predict(X_test), y_test)))


**Comparando o desempenho do modelo**

In [ ]:
df_score = pd.DataFrame(list(zip(name,rmse, score, models)),columns=['name','rmse','score',"model"])
df_score.set_index('name',inplace=True)
df_score.sort_values(by=['score'],inplace=True)
df_score

**Previsão**

In [ ]:
model = df_score.loc["XGBRegressor","model"]
data_test.drop(['Weekly_Sales'],axis=1,inplace=True)
predict = model.predict(data_test)
predict